# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("output.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,boa vista,2.8197,-60.6733,73.38,94,75,8.05,1970-01-01,BR
1,1,tuktoyaktuk,69.4541,-133.0374,68.00,64,75,13.80,1970-01-01,CA
2,2,pangody,65.8500,74.4859,62.37,57,100,9.51,1970-01-01,RU
3,3,kerema,-7.9631,145.7785,76.57,83,94,4.94,1970-01-01,PG
4,4,saskylakh,71.9167,114.0833,68.20,44,0,7.49,1970-01-01,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
# Create initial figure
fig = gmaps.figure()

# Create Heatmap layer
locations = weather_df[["Lat", "Lng"]]
heatmap_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"])
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create good weather conditions for holiday
idx = np.where((weather_df['Wind Speed']< 10)&
               (weather_df['Max Temp']>70)&
               (weather_df['Max Temp']<80)&
               (weather_df['Cloudiness']==0))


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#store in hotel_df dataframe
hotel_df = weather_df.loc[idx]
#create new column
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
76,76,rongcheng,30.6340,117.8517,78.75,91,0,2.39,1970-01-01,CN,
197,197,guangyuan,32.4420,105.8230,78.85,87,0,1.34,1970-01-01,CN,
223,223,mianyang,31.4593,104.7542,79.79,74,0,2.42,1970-01-01,CN,
299,299,yonezawa,37.9167,140.1167,74.12,96,0,1.92,1970-01-01,JP,
360,360,ochamchira,42.7121,41.4721,74.88,86,0,1.48,1970-01-01,GE,
379,379,eenhana,-17.4667,16.3333,76.75,23,0,5.39,1970-01-01,NaN,
501,501,ambilobe,-13.2000,49.0500,71.82,74,0,3.62,1970-01-01,MG,
503,503,urumqi,43.8010,87.6005,70.75,38,0,4.47,1970-01-01,CN,
568,568,jinan,36.6683,116.9972,76.26,85,0,4.09,1970-01-01,CN,


In [6]:
#copy lat and longitude to a list zipped together
lat =hotel_df['Lat'].to_list()
lng = hotel_df['Lng'].to_list()
lat_lng = list(zip(lat, lng))

#create variables for params
target_coordinates = lat_lng
target_radius = 5000
target_type = "hotel"

#set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

#requesting the hotels and saving the first hotel in hotel name
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel_address = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/Result ... skipping")
    
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
76,76,rongcheng,30.6340,117.8517,78.75,91,0,2.39,1970-01-01,CN,Chizhou
197,197,guangyuan,32.4420,105.8230,78.85,87,0,1.34,1970-01-01,CN,Guangyuan
223,223,mianyang,31.4593,104.7542,79.79,74,0,2.42,1970-01-01,CN,Mianyang
299,299,yonezawa,37.9167,140.1167,74.12,96,0,1.92,1970-01-01,JP,Yonezawa
360,360,ochamchira,42.7121,41.4721,74.88,86,0,1.48,1970-01-01,GE,Ochamchire
379,379,eenhana,-17.4667,16.3333,76.75,23,0,5.39,1970-01-01,NaN,Eenhana
501,501,ambilobe,-13.2000,49.0500,71.82,74,0,3.62,1970-01-01,MG,Ambilobe
503,503,urumqi,43.8010,87.6005,70.75,38,0,4.47,1970-01-01,CN,Ürümqi
568,568,jinan,36.6683,116.9972,76.26,85,0,4.09,1970-01-01,CN,Jinan


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

marker_locations = hotel_df[['Lat', 'Lng']]
hotel_name = hotel_df["Hotel Name"].tolist()


#create two layers
heatmap_layer = gmaps.heatmap_layer(marker_locations, weights=hotel_df["Humidity"])
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))